# Test of GNN models

In [1]:
import dgl
import cogdl
import pickle
import numpy as np
import networkx as nx
import scipy.sparse as sp
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt

Using backend: pytorch


In [2]:
import sys
sys.path.append('..')

# Load data

In [13]:
from grb.dataset.dataloader import DataLoader

In [14]:
dataset = DataLoader('cora', mode='normal')

Dataset 'cora' loaded.
    Number of nodes: 2708.
    Number of edges: 5242.
    Number of features: 1433.
    Number of classes: 7.
    Number of train samples: 140.
    Number of val samples: 500.
    Number of test samples: 1000.
    Feature range: [0.0000, 1.0000]


In [15]:
adj = dataset.adj
adj_tensor = dataset.adj_tensor
features = dataset.features
labels = dataset.labels
num_features = dataset.num_features
num_classes = dataset.num_classes

In [6]:
from grb.utils import fix_seed, get_num_params

fix_seed(seed=0)

# GCN test

In [7]:
from grb.model.gcn import GCN

model = GCN(in_features=num_features, out_features=num_classes, hidden_features=[16], activation=F.relu)
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)

Number of parameters: 23335.
GCN(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=1433, out_features=16, bias=True)
    )
    (1): GCNConv(
      (linear): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (linear): Linear(in_features=16, out_features=7, bias=True)
    )
  )
)


## Model training

In [8]:
from grb.model.trainer import Trainer
from grb.utils.normalize import GCNAdjNorm

adam = torch.optim.Adam(model.parameters(), lr=0.01)
kwargs = {'adj_norm_func': GCNAdjNorm}
trainer = Trainer(dataset=dataset, optimizer=adam, loss=F.nll_loss, device='cpu', **kwargs)

In [9]:
model_name = 'model_gcn_cora.pt'
trainer.set_config(n_epoch=200, eval_every=10, save_path='../grb/model/saved_models/' + model_name)
model.train()

GCN(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=1433, out_features=16, bias=True)
    )
    (1): GCNConv(
      (linear): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (linear): Linear(in_features=16, out_features=7, bias=True)
    )
  )
)

In [10]:
trainer.train(model, dropout=0.5)

Epoch 00000 | Loss 1.9597 | Train Acc 0.1429 | Val Acc 0.0580
Epoch 00010 | Loss 1.9290 | Train Acc 0.1929 | Val Acc 0.1940
Epoch 00020 | Loss 1.8534 | Train Acc 0.3357 | Val Acc 0.2580
Epoch 00030 | Loss 1.6814 | Train Acc 0.4143 | Val Acc 0.2620
Epoch 00040 | Loss 1.4250 | Train Acc 0.5214 | Val Acc 0.3300
Epoch 00050 | Loss 1.1388 | Train Acc 0.5929 | Val Acc 0.4140
Epoch 00060 | Loss 0.9187 | Train Acc 0.6786 | Val Acc 0.4760
Epoch 00070 | Loss 0.7197 | Train Acc 0.7857 | Val Acc 0.5160
Epoch 00080 | Loss 0.6497 | Train Acc 0.7857 | Val Acc 0.5540
Epoch 00090 | Loss 0.5425 | Train Acc 0.8357 | Val Acc 0.5620
Epoch 00100 | Loss 0.4783 | Train Acc 0.8286 | Val Acc 0.6240
Epoch 00110 | Loss 0.4179 | Train Acc 0.8786 | Val Acc 0.6280
Epoch 00120 | Loss 0.3253 | Train Acc 0.9071 | Val Acc 0.6160
Epoch 00130 | Loss 0.2559 | Train Acc 0.9429 | Val Acc 0.6260
Epoch 00140 | Loss 0.2715 | Train Acc 0.9286 | Val Acc 0.6260
Epoch 00150 | Loss 0.2506 | Train Acc 0.9214 | Val Acc 0.6520
Epoch 00

## Model inference

In [11]:
from grb.utils import evaluator

model.eval()
pred = model.forward(features, adj_tensor, dropout=0)
pred_label = torch.argmax(pred, dim=1)
acc = evaluator.eval_acc(pred, labels, mask=dataset.test_mask)
print("Test accuracy: {:.4f}".format(acc))

Test accuracy: 0.7060


# GAT test

In [12]:
from grb.model.gat import GAT

model = GAT(num_layers=3, num_heads=, head_dim, activation=F.leaky_relu)
print("Number of parameters: {}.".format(get_num_params(model)))
print(model)

TypeError: __init__() got an unexpected keyword argument 'in_features'